## 불균형한 클래스 다루기

클래스 불균형은 한 개 또는 여러 개의 클래스 샘플이 데이터셋에 너무 많은 경우이다.</br>
만약 유방암 데이터의 90%가 건강한 환자라면 전부 양성으로 찍어도 90% 정확도를 얻을 수 있다.</br>
이 경우, 분류기 모델을 통해서 90%를 달성해도 유용한 결과를 얻지 못한 것이다.</br>
이와같은 데이터셋은 모델의 성능 평가 지표로 정확도는 적합하지 않다.</br>
주요 관심대상에 따라서 정밀도, 재현율, ROC 곡선 등을 사용할 수 있다.</br>

클래스 불균형은 모델이 훈련되는 동안 학습 알고리즘 훈련에 영향을 미친다.</br>
훈련 과정에서 비용 최소화나 보상 최대화를 위해, 가장 빈도높은 클래스의 예측을 최적화한다.</br>

불균형한 클래스를 다루는 방법은 다음과 같다. 

- 해당 클래스의 샘플의 비중에 따른 가중치를 부여한다.
- 소스 클래스 샘플을 늘리거나 다수 클래스 샘플을 줄인다.
- 인공적으로 훈련 샘플을 생성한다(SMOTE 알고리즘).

다음은 사이킷런을 사용해서 불균형한 유방암 데이터셋을 만들고, 소수 클래스를 늘리는 과정을 구현한 것이다.

In [9]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

df = pd.read_csv(   # 위스콘신 유방암 데이터셋 로딩
    'https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/wdbc.data', 
    header=None
)

x = df.loc[:, 2:].values    # 30개의 특성을 x에 할당
y = df.loc[:, 1].values     # 클래스 레이블을 y에 할당
y = LabelEncoder().fit_transform(y) # 클래스 레이블을 정수로 변환(악성: 1, 양성: 0)

# 불균형한 데이터셋으로 만든다.(40개 악성 종양, 357개 양성 종양)
x_imb = np.vstack((x[y == 0], x[y == 1][:40]))
y_imb = np.hstack((y[y == 0], y[y == 1][:40]))
y_pred = np.zeros(y_imb.shape[0])
# 약 90%가 양성 종양 샘플인 것을 확인한다.
print(np.mean(y_pred == y_imb) * 100)

89.92443324937027


In [12]:
from sklearn.utils import resample

print('샘플링하기 전 악성 종양 샘플 개수: ', x_imb[y_imb == 1].shape[0])

# resample 함수는 중복을 허용한 추출 방식으로 소수 클래스의 샘플을 늘리는데 사용된다.
x_unsampled, y_unsampled = resample(
    x_imb[y_imb == 1],  # 클래서 레이블을 0으로 바꾸면 다운 샘플링된다.
    y_imb[y_imb == 1],
    replace=True,
    n_samples=x_imb[y_imb == 0].shape[0],
    random_state=123
)

print('샘플링한 후 악성 종양 샘플 개수: ', x_unsampled.shape[0])

샘플링하기 전 악성 종양 샘플 개수:  40
샘플링한 후 악성 종양 샘플 개수:  357


In [13]:
# 양성인 원본 샘플과 샘플린된 음성 샘플을 연결해서 균형 잡힌 데이터셋을 만든다. 
x_bal = np.vstack((x[y==0], x_unsampled))
y_bal = np.hstack((y[y==0], y_unsampled))
y_pred = np.zeros(y_bal.shape[0])
# 양성과 음성 샘플의 비율이 5:5가 된것을 확인
print(np.mean(y_pred == y_bal) * 100)

50.0


유사하게 데이터셋에서 다수 클래스 샘플을 삭제해서 다운샘플링할 수 있다.